In [ ]:
import os
import numpy as np
import csv

def bin_all_frames_to_iq_csv(bin_file_path,
                             csv_file_path,
                             num_samples_per_chirp,
                             num_chirps_per_frame,
                             num_antennas,
                             dtype=np.int16):
    
    print(f"Reading: {bin_file_path}")

    adc_data = np.fromfile(bin_file_path, dtype=dtype)
    total_values = adc_data.size

    #Ensure even iq pairs
    if total_values % 2 != 0:
        adc_data = adc_data[:-1]
        total_values -= 1

    total_complex = total_values // 2

    samples_per_frame = num_samples_per_chirp * num_chirps_per_frame * num_antennas
    num_full_frames = total_complex // samples_per_frame

    usable_values = num_full_frames * samples_per_frame * 2
    adc_data = adc_data[:usable_values]  

    print(f"Using {num_full_frames} full frames")

    adc_data = adc_data.reshape((num_full_frames,
                                 num_chirps_per_frame,
                                 num_samples_per_chirp,
                                 num_antennas,
                                 2))

    with open(csv_file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Frame Index', 'Chirp Index', 'Sample Index', 'Antenna Index', 'I', 'Q'])

        #loop over bin files, write to csv
        for frame in range(num_full_frames):
            for chirp in range(num_chirps_per_frame):
                for sample in range(num_samples_per_chirp):
                    for antenna in range(num_antennas):
                        i_val = adc_data[frame, chirp, sample, antenna, 0]
                        q_val = adc_data[frame, chirp, sample, antenna, 1]
                        writer.writerow([frame, chirp, sample, antenna, int(i_val), int(q_val)])

    print(f"CSV saved to: {csv_file_path}")


bin_file_path = r"E:\Masters Project\Raw Data\Raw Data Collection\Subject 1 - Run 1 - Sitting - 4 Minutes\adc_data_Raw_0.bin"
csv_file_path = r"E:\Masters Project\Raw Data\Raw Data Collection\Subject 1 - Run 1 - Sitting - 4 Minutes\convertedIQ.csv"

num_samples_per_chirp = 256
num_chirps_per_frame  = 64 
num_antennas          = 6

bin_all_frames_to_iq_csv(bin_file_path, csv_file_path,
                         num_samples_per_chirp,
                         num_chirps_per_frame,
                         num_antennas)
